In [109]:
#Add all the imports here
import yfinance as yf
from datetime import datetime, timedelta, time
import pandas as pd
import numpy as np
from itertools import combinations
import itertools
import plotly.graph_objects as go
import time


In [110]:
#To get data finanace
def get_data(symbol, date,interval):
  ticker = yf.Ticker(symbol)
  start_date = datetime.strptime(date, "%Y-%m-%d")
  end_date = start_date + timedelta(days=1)
  try:
    data = ticker.history(start=start_date, end=end_date, interval=interval);
  except Exception as e:
    return None
  return data

In [111]:
#Graph helpers

def create_candlestick_figure(ticker_symbol,date):
    fig = go.Figure()

    fig.update_layout(
        title=f'{ticker_symbol} Candlestick Chart',
        xaxis_title=f"Date:{date}",
        yaxis_title="Price",
        dragmode="pan",                 # ✅ allow panning
        xaxis=dict(
            type="category",
            rangeslider=dict(visible=False),
            tickmode="array",
            showticklabels=False,  # ✅ hides x-axis labels
        )
    )

    # Enable zoom in both directions
    fig.update_xaxes(fixedrange=False)
    fig.update_yaxes(fixedrange=False)

    return fig


def update_candlestick_data(
    fig: go.Figure,
    historical_data: pd.DataFrame
):
    candle = go.Candlestick(
        x=historical_data.index,
        open=historical_data["Open"],
        high=historical_data["High"],
        low=historical_data["Low"],
        close=historical_data["Close"],
        increasing_line_color="green",
        decreasing_line_color="red",
    )
    fig.add_trace(candle)
    return fig

def add_hline(fig: go.Figure,value,annot,color="blue",annot_pos="top left"):
    fig.add_hline(
      y= value,
      line=dict(color=color, width=1, dash="dash"),
      annotation_text=f"{annot}",
      annotation_position=annot_pos
    )
    return fig

def add_vline(fig: go.Figure,date):
    start_dt = pd.Timestamp(f"{date} 13:10:00").tz_localize("Asia/Kolkata")
    end_dt   = pd.Timestamp(f"{date} 13:15:00").tz_localize("Asia/Kolkata")

    fig.add_vrect(
      x0=start_dt,
      x1=end_dt,
      fillcolor="rgba(255,0,0,0.12)",  # translucent red band
      line_width=0,
      layer="below"
    )
    return fig


def add_marker(fig: go.Figure,time,price,text):
    fig.add_annotation(
      x=time,
      y=price,
      text=text,
      showarrow=True,
      arrowhead=2,
      arrowsize=1.2,
      arrowcolor="crimson",
      ax=-40, ay=-40,
      bgcolor="rgba(255,255,255,0.8)",
      bordercolor="crimson",
      borderwidth=1,
      font=dict(size=12, color="black"),
      xref="x", yref="y"
    )
    return fig



In [112]:
#Strategy1_testing

def strategy1(ticker_symbol, date):
  fig = create_candlestick_figure(ticker_symbol,date)
  data_4h = get_data(ticker_symbol, date, "4h");

  high_4h = data_4h["High"].iloc[0]
  low_4h = data_4h["Low"].iloc[0]
  end_4h = data_4h.index[1]

  data = get_data(ticker_symbol, date, "5m");

  fig = update_candlestick_data(fig, historical_data=data)
  fig = add_hline(fig,high_4h,"High_4h")
  fig = add_hline(fig,low_4h,"Low_4h")
  fig = add_vline(fig,date)

  start_tf = pd.Timestamp(f"{date} 13:15:00").tz_localize("Asia/Kolkata")

  flag_breakout_short = False;
  flag_breakout_long = False;

  flag_entry_short = False;
  flag_entry_long = False;

  short_risk = float("-inf")
  long_risk  = float("inf")

  for row in data.itertuples():
    index = row.Index
    if index < start_tf:
        continue

    close = row.Close
    high  = row.High
    low   = row.Low

    # ----- SHORT LOGIC -----
    if not flag_breakout_short:
      if close > high_4h:
        flag_breakout_short = True;
        short_risk = high
        fig =add_marker(fig,index,close,"Breakout")
    else:
      short_risk = max(short_risk, high)
      if close < high_4h:
        fig = add_marker(fig, index, close, "Entry(Short)")
        fig = add_hline(fig,short_risk,"Stoploss","red","top right")
        diff = close - short_risk
        target = close + (diff * 2)
        fig = add_hline(fig,target,"target","green","top right")
        flag_entry_short = True;
        break;

    # ----- LONG LOGIC -----
    if not flag_breakout_long:
      if close < low_4h:
        flag_breakout_long = True;
        long_risk = low
        fig = add_marker(fig, index, close, "Breakout")
    else:
      long_risk = min(long_risk, low)
      if close > low_4h:
        fig = add_marker(fig, index, close, "Entry(Long)")
        fig = add_hline(fig,long_risk,"Stoploss","red", "top right")
        diff = long_risk - close
        target = close - (diff * 2)
        fig = add_hline(fig,target,"target","green","top right")
        flag_entry_long = True;
        break;

  return fig


In [113]:
tz="Asia/Kolkata"

def run_strategy_range(ticker_symbol, days_back, pause_each=True):
    today = pd.Timestamp.now(tz=tz).normalize()
    start = today - pd.Timedelta(days=days_back)
    dates = pd.date_range(start=start, end=today, freq="D", tz=tz)

    total = len(dates)
    print(f"Running from {dates[0].date()} to {dates[-1].date()} ({total} days)…")

    auto = False
    for i, dt in enumerate(dates, start=1):
        date_str = dt.strftime("%Y-%m-%d")  # matches your strategy1 signature
        print(f"\n[{i}/{total}] strategy1({ticker_symbol!r}, {date_str})")

        try:
            fig = strategy1(ticker_symbol, date_str)  # <-- your existing function
            fig.show(config={"scrollZoom": True})
        except Exception as e:
            #print(f"⚠️ Error for {date_str}: {e}")
            pass



In [114]:
#Main entry
ticker_symbol = "BTC-USD"
date = "2025-12-19"

run_strategy_range(ticker_symbol,20)
#fig = strategy1(ticker_symbol, date);
#fig.show(config={"scrollZoom": True})


Running from 2025-12-02 to 2025-12-22 (21 days)…

[1/21] strategy1('BTC-USD', 2025-12-02)



[2/21] strategy1('BTC-USD', 2025-12-03)



[3/21] strategy1('BTC-USD', 2025-12-04)



[4/21] strategy1('BTC-USD', 2025-12-05)



[5/21] strategy1('BTC-USD', 2025-12-06)


...


[17/21] strategy1('BTC-USD', 2025-12-18)



[18/21] strategy1('BTC-USD', 2025-12-19)



[19/21] strategy1('BTC-USD', 2025-12-20)



[20/21] strategy1('BTC-USD', 2025-12-21)



[21/21] strategy1('BTC-USD', 2025-12-22)
